# Tube commute
Data analysis of the London Underground journey history based on the Oyster card data from the [Transport For London website](https://oyster.tfl.gov.uk/oyster/journeyHistory.do).
You can subscribe and get files containing all the trip details periodically emailed to you.

The goal of this project is to provide an easy way to explore TFL data. Once you loaded the script in R you can play with it interactively. An example session could look like:

In [ ]:
source("./commute.r")

# Preparation of the data

In [ ]:
tfl_data <- load_folder('~/Documents/Data/TFL/')
home <- "Tottenham Hale [London Underground]"
work <- "Warren Street"
commute.to <- extract_commute_leg(tfl_data, home, work, 7, 60*3)
commute.from <- extract_commute_leg(tfl_data, work, home, 16, 60*3)
commute.labeled <- merge_labeled(commute.to, commute.from)

In [ ]:
commute.to <- commute.to[order(commute.to$Date),]
commute.from <- commute.from[order(commute.from$Date),]
commute.labeled <- commute.labeled[order(commute.labeled$Date),]

In [ ]:
head(commute.labeled)
summary(commute.labeled)

# Visualizations

In [ ]:
start_time_from_vs_to <- function(commute.labeled){
    chart <- ggplot(commute.labeled, aes(x=Date, y=(Start.Time/60), colour= Direction)) +
        geom_point() +
        geom_smooth(method='lm',formula=y~x, se=FALSE)    
        scale_y_discrete(name="Ride start time (hour of the day)", limits=7:19)
    return(chart)
}
start_time_from_vs_to(commute.labeled)

In [ ]:
visualisation_from_vs_to <- function(commute.labeled){
    hist <- ggplot(commute.labeled, aes(x=Duration, color= Direction)) +
        geom_density() +
        xlab("Ride duration (minutes)") +
        ylab("Fraction of occurrences")
    return(hist)
}
visualisation_from_vs_to(commute.labeled)

In [ ]:
visualisation_from_vs_to_by_day <- function(commute.data){
    hist <- ggplot(commute.data, aes(x=Duration, color= Direction)) +
        geom_density() +
        facet_grid(Weekday ~.) +
        xlab("Ride duration (minutes)") +
        ylab("Fraction of occurrences")
    return(hist)
}
visualisation_from_vs_to_by_day(commute.labeled)

In [ ]:
visualise_time_vs_duration(commute.labeled)